In [1]:
#import standard lib
import pandas as pd
import numpy as np
from datetime import datetime
import re
import matplotlib.pylab as plt
from math import sqrt
import os
import seaborn as sns

from matplotlib.pyplot import rcParams 
rcParams['figure.figsize']=15,6 

# preprocessing and split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# import models
import xgboost as xgb
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import GridSearchCV
#Error metrics
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_csv('c:/Users/ThinkPad/Desktop/rohdaten_v4.csv',float_precision='round_trip')


#d1=d[d['timestamp'].str.contains((r'2020-05-14'))]

#d1=df['2020-05-01':'2020-06-15']

#d1.to_csv('05.csv')
print(df)


        vehiclecount            timestamp  node  co_2_esp  co_op1  co_op2  \
0             1999.0  2020-02-19 00:00:10    12      2030    6206    5416   
1             1999.0  2020-02-19 00:00:40    12      2004    6205    5401   
2             2000.0  2020-02-19 00:01:10    12      1972    6213    5390   
3             2001.0  2020-02-19 00:01:40    12      2030    6247    5403   
4             2002.0  2020-02-19 00:02:10    12      2158    6305    5391   
...              ...                  ...   ...       ...     ...     ...   
521396           NaN  2020-11-06 23:57:39    12      2074    6382    5234   
521397           NaN  2020-11-06 23:58:09    12      2025    6384    5253   
521398           NaN  2020-11-06 23:58:39    12      1962    6447    5256   
521399           NaN  2020-11-06 23:59:09    12      2230    6345    5234   
521400           NaN  2020-11-06 23:59:39    12      1684    6396    5289   

        counter_arduino  counter_esp  ir_brightness  lrvehiclecount  ...  \

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521401 entries, 0 to 521400
Data columns (total 40 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   vehiclecount        346327 non-null  float64
 1   timestamp           521401 non-null  object 
 2   node                521401 non-null  int64  
 3   co_2_esp            521401 non-null  int64  
 4   co_op1              521401 non-null  int64  
 5   co_op2              521401 non-null  int64  
 6   counter_arduino     521401 non-null  int64  
 7   counter_esp         521401 non-null  int64  
 8   ir_brightness       521401 non-null  int64  
 9   lrvehiclecount      346327 non-null  float64
 10  no_2_op1            521401 non-null  int64  
 11  no_2_op2            521401 non-null  int64  
 12  o_3_op1             521401 non-null  int64  
 13  o_3_op2             521401 non-null  int64  
 14  pm1                 520883 non-null  float64
 15  pm10                520883 non-nul

In [4]:
import missingno as msno
import datetime


data=df[["no_2_op1","no_2_op2","temperature_opc",'timestamp']]
data['datetime'] =data['timestamp']
data['timestamp'] = pd.to_datetime(data['timestamp'])
data = data.set_index('timestamp') 
data.head()

,no_2_op1,no_2_op2,temperature_opc,datetime
timestamp,,,,
2020-02-19 00:00:10,3815,3748,9.13795,2020-02-19 00:00:10
2020-02-19 00:00:40,3808,3736,9.16347,2020-02-19 00:00:40
2020-02-19 00:01:10,3814,3743,9.16347,2020-02-19 00:01:10
2020-02-19 00:01:40,3795,3740,9.20501,2020-02-19 00:01:40
2020-02-19 00:02:10,3820,3739,9.20501,2020-02-19 00:02:10


In [6]:
data.no_2_op1 = data.no_2_op1*(2048.0 / 32768.0)
data.no_2_op2= data.no_2_op2*(2048.0 / 32768.0)
def nT(temp):
    if (temp < 0):
        nTa=1.3
    else:
        if (temp < 10):
            nTa=1.3 - 0.03*temp
        else:
            if (temp < 20):
                nTa=1.0 - 0.04*(temp - 10)
            else:
                if (temp < 30):
                    nTa=0.6 - 0.02*(temp - 20)
                else:
                    if (temp < 40):
                        nTa=0.4 - 0.02*(temp - 30)
                    else:
                        if (temp < 50):
                            nTa=0.2 - 0.17*(temp - 40)
                        else:
                            nTa=- 1.5
    
    return nTa

def nTSens(temp):
    if (temp < - 30):
        nTa=101
    else:
        if (temp < - 20):
            nTa=101 - 1.2*(temp + 30)
        else:
            if (temp < - 10):
                nTa=89 - 0.6*(temp + 20)
            else:
                if (temp < 0):
                    nTa=83 + 0.4*(temp + 10)
                else:
                    if (temp < 10):
                        nTa=87 + 0.6*temp
                    else:
                        if (temp < 20):
                            nTa=93 + 0.7*(temp - 10)
                        else:
                            if (temp < 30):
                                nTa=100 + 0.5*(temp - 20)
                            else:
                                if (temp < 40):
                                    nTa=105 + 0.3*(temp - 30)
                                else:
                                    if (temp < 50):
                                        nTa=108 + 0.3*(temp - 40)
                                    else:
                                        nTa=111
    nTa=nTa / 100.0
    return nTa
    
s=data.temperature_opc
data['nT']=s.apply(lambda x=None: nT(x))
data['nTSens']=s.apply(lambda x=None: nTSens(x))
data.head()   

,no_2_op1,no_2_op2,temperature_opc,datetime,nT,nTSens
timestamp,,,,,,
2020-02-19 00:00:10,238.4375,234.2500,9.13795,2020-02-19 00:00:10,1.025861,0.924828
2020-02-19 00:00:40,238.0000,233.5000,9.16347,2020-02-19 00:00:40,1.025096,0.924981
2020-02-19 00:01:10,238.3750,233.9375,9.16347,2020-02-19 00:01:10,1.025096,0.924981
2020-02-19 00:01:40,237.1875,233.7500,9.20501,2020-02-19 00:01:40,1.023850,0.925230
2020-02-19 00:02:10,238.7500,233.6875,9.20501,2020-02-19 00:02:10,1.023850,0.925230


In [7]:
data.no_2_op1=data.no_2_op1-238
data.no_2_op2=data.no_2_op2-235
sens_cert=0.283
factor=1.91
data['no2_ppb']=(data.no_2_op1-data.nT*data.no_2_op2)/(data['nTSens']*sens_cert)
data['no2']=(data.no_2_op1-data.nT*data.no_2_op2)/(data['nTSens']*sens_cert)*factor
data.head()

                        

,no_2_op1,no_2_op2,temperature_opc,datetime,nT,nTSens,no2_ppb,no2
timestamp,,,,,,,,
2020-02-19 00:00:10,0.4375,-0.7500,9.13795,2020-02-19 00:00:10,1.025861,0.924828,4.611292,8.807568
2020-02-19 00:00:40,0.0000,-1.5000,9.16347,2020-02-19 00:00:40,1.025096,0.924981,5.874036,11.219408
2020-02-19 00:01:10,0.3750,-1.0625,9.16347,2020-02-19 00:01:10,1.025096,0.924981,5.593333,10.683266
2020-02-19 00:01:40,-0.8125,-1.2500,9.20501,2020-02-19 00:01:40,1.023850,0.925230,1.784723,3.408821
2020-02-19 00:02:10,0.7500,-1.3125,9.20501,2020-02-19 00:02:10,1.023850,0.925230,7.996494,15.273303


In [10]:
no2=data[['no2']]
no2[no2<0]= np.int(0)

In [11]:
no2.to_csv("no2.csv",float_format='%.3f')

In [12]:
no2.head()

,no2
timestamp,
2020-02-19 00:00:10,8.807568
2020-02-19 00:00:40,11.219408
2020-02-19 00:01:10,10.683266
2020-02-19 00:01:40,3.408821
2020-02-19 00:02:10,15.273303


In [15]:
mean_1h=no2.resample('60min').mean()
mean_1h['no2']=mean_1h
mean_1h.head()
mean_1h.to_csv("no2_mean_1h.csv",float_format='%.3f')